## Assignment 1
### Name: Subhash Agrawal
### Roll Number: 23CS60R67

In [1]:
# import all the necessary libraries here
import pandas as pd
import numpy as np

In [2]:
#Import Dataset
df = pd.read_csv('../../dataset/cross-validation.csv')
print(df.shape) 

(614, 13)


In [3]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


# Preprocessing Data

In [4]:
# Import the necessary modules
import pandas as pd

# Define the features and the target
X = df.drop(['Loan_ID','Loan_Status'], axis=1)
y = df['Loan_Status']

# Encode the categorical features
X = pd.get_dummies(X)

# Handle the missing values
X = X.fillna(X.mean())

#Standardize Data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

print(X)
X.shape, y.shape

[[ 0.07299082 -0.55448733  0.         ... -0.64147818 -0.7820157
   1.42814704]
 [-0.13441195 -0.03873155 -0.21927331 ...  1.55889948 -0.7820157
  -0.70020801]
 [-0.39374734 -0.55448733 -0.957641   ... -0.64147818 -0.7820157
   1.42814704]
 ...
 [ 0.43717437 -0.47240418  1.26937121 ... -0.64147818 -0.7820157
   1.42814704]
 [ 0.35706382 -0.55448733  0.4833669  ... -0.64147818 -0.7820157
   1.42814704]
 [-0.13441195 -0.55448733 -0.15972753 ... -0.64147818  1.2787467
  -0.70020801]]


((614, 20), (614,))

# Logistic Regression Model

In [5]:
# Create a logistic regression object with saga solver, no penalty, and increased max_iter
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver='saga', penalty=None, max_iter=10000)


# Fit the model to the data
log_reg.fit(X,y)

# Print the model coefficients and intercept
print('Coefficients:', log_reg.coef_)
print('Intercept:', log_reg.intercept_)


Coefficients: [[ 0.06008844 -0.14028762 -0.14113604 -0.0862345   1.3554325   0.11215183
   0.09677877 -1.76207326 -1.48139026  0.05184501 -0.12581014  0.13320622
   0.02817968  0.07992762 -0.07992762  0.02690111  0.02009163 -0.17934609
   0.24113104 -0.07556435]]
Intercept: [0.86987516]


# k-fold Cross Validation

In [6]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Define the number of folds
num_folds = 5
fold_size = len(X) // num_folds


In [7]:
# Initialize lists to store evaluation metrics
accuracy_scores = []
precision_scores = []
recall_scores = []

# Perform k-fold cross-validation
for i in range(num_folds):
    # Split the data into training and testing sets for this fold
    start_idx = i * fold_size
    end_idx = (i + 1) * fold_size

    X_test_fold = X[start_idx:end_idx]
    y_test_fold = y[start_idx:end_idx]

    X_train_fold = np.concatenate((X[:start_idx], X[end_idx:]))
    y_train_fold = np.concatenate((y[:start_idx], y[end_idx:]))

    # Create and fit the logistic regression model
    log_reg = LogisticRegression(solver='saga', penalty='none', max_iter=10000)
    log_reg.fit(X_train_fold, y_train_fold)

    # Make predictions on the test fold
    y_pred_fold = log_reg.predict(X_test_fold)

    conf_matrix = confusion_matrix(y_test_fold, y_pred_fold)
    print("Confusion Matrix for Fold - ",i,"\n",conf_matrix,"\n")

    # Calculate and store evaluation metrics for this fold
    accuracy = np.mean(y_pred_fold == y_test_fold)
    true_positives = np.sum((y_pred_fold == 'Y') & (y_test_fold == 'Y'))
    false_positives = np.sum((y_pred_fold == 'Y') & (y_test_fold == 'N'))
    false_negatives = np.sum((y_pred_fold == 'N') & (y_test_fold == 'Y'))

    if true_positives + false_positives == 0:
        precision = 0  # Handle division by zero
    else:
        precision = true_positives / (true_positives + false_positives)

    if true_positives + false_negatives == 0:
        recall = 0  # Handle division by zero
    else:
        recall = true_positives / (true_positives + false_negatives)

    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)

print("Accuracy Score Vector - ",accuracy_scores)
print("Precision Score Vector - ",precision_scores)
print("Recall Score Vector - ",recall_scores)

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


Confusion Matrix for Fold -  0 
 [[17 23]
 [ 1 81]] 



C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


Confusion Matrix for Fold -  1 
 [[13 25]
 [ 4 80]] 



C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


Confusion Matrix for Fold -  2 
 [[13 22]
 [ 3 84]] 



C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


Confusion Matrix for Fold -  3 
 [[22 17]
 [ 1 82]] 



C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


Confusion Matrix for Fold -  4 
 [[18 21]
 [ 3 80]] 

Accuracy Score Vector -  [0.8032786885245902, 0.7622950819672131, 0.7950819672131147, 0.8524590163934426, 0.8032786885245902]
Precision Score Vector -  [0.7788461538461539, 0.7619047619047619, 0.7924528301886793, 0.8282828282828283, 0.7920792079207921]
Recall Score Vector -  [0.9878048780487805, 0.9523809523809523, 0.9655172413793104, 0.9879518072289156, 0.963855421686747]


# Average Accuracy, Precision & Recall

In [8]:

# Calculate mean scores across all folds
mean_accuracy = np.mean(accuracy_scores)
mean_precision = np.mean(precision_scores)
mean_recall = np.mean(recall_scores)

# Print the mean scores
print("Mean Accuracy:", mean_accuracy)
print("Mean Precision (for class 1):", mean_precision)
print("Mean Recall (for class 1):", mean_recall)


Mean Accuracy: 0.8032786885245902
Mean Precision (for class 1): 0.790713156428643
Mean Recall (for class 1): 0.971502060144941
